In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px

class LiquidityReporter:
    def __init__(self, balance_sheet_path: str):
        self.df = pd.read_csv(balance_sheet_path, parse_dates=['date'])
        self.validate_data()
        self.df['hqla'] = self.df['cash_treasuries'] + self.df['central_bank_reserves']
        self.df['outflows'] = self.df['deposit_outflows_30d'] + self.df['wholesale_funding_runoff']
        self.df['deposits'] = self.df['stable_deposits']
        
    def validate_data(self):
        """Check for missing values and negative balances."""
        assert not self.df.isnull().values.any(), "Missing values detected"
        assert (self.df.select_dtypes(include=[np.number]) >= 0).all().all(), "Negative balances invalid"

    def calculate_lcr(self) -> float:
        """Liquidity Coverage Ratio (Basel III)"""
        hqla = self.df['cash_treasuries'] + self.df['central_bank_reserves']
        net_outflows = self.df['deposit_outflows_30d'] + self.df['wholesale_funding_runoff']
        return (hqla.sum() / net_outflows.sum()) * 100

    def calculate_nsfr(self) -> float:
        """Net Stable Funding Ratio"""
        asf = (self.df['stable_deposits'] * 0.95 +
               self.df['tier1_capital'] * 1.0)
        rsf = (self.df['loans_1y'] * 0.85 +
               self.df['mortgages'] * 0.65)
        return (asf.sum() / rsf.sum()) * 100

    def generate_fr2052a_report(self) -> pd.DataFrame:
        """FR 2052a-style liquidity report"""
        return pd.pivot_table(
            self.df,
            values=['cash_flows', 'liabilities'],
            index=['date'],
            columns=['product_type'],
            aggfunc=np.sum
        )

class StressTester:
    SCENARIOS = {
        "Baseline": {"deposit_runoff": 0.05, "haircut": 0.03},
        "Severe": {"deposit_runoff": 0.20, "haircut": 0.15}
    }

    def __init__(self, balance_sheet_path: str):
        self.reporter = LiquidityReporter(balance_sheet_path)

    def run_scenarios(self) -> pd.DataFrame:
        results = []
        for name, params in self.SCENARIOS.items():
            stressed_df = self.reporter.df.copy()
            stressed_df['deposits'] *= (1 - params['deposit_runoff'])
            stressed_df['hqla'] *= (1 - params['haircut'])
            lcr = (stressed_df['hqla'].sum() /
                   stressed_df['outflows'].sum()) * 100
            results.append({"Scenario": name, "LCR": lcr})
        return pd.DataFrame(results)

    def plot_gaps(self):
        df = self.run_scenarios()
        fig = px.bar(
            df, x="Scenario", y="LCR",
            title="LCR Under Stress Scenarios",
            labels={"LCR": "LCR (%)"},
            color="LCR",
            color_continuous_scale="RdYlGn"
        )
        fig.write_html("dashboards/stress_test.html")

StressTester("jpm_10k_2023.csv").plot_gaps()

from statsmodels.tsa.arima.model import ARIMA
import pandas as pd

class BalanceSheetForecaster:
    def __init__(self, data_path: str):
        self.df = pd.read_csv(data_path, index_col='date', parse_dates=True)

    def forecast_13q(self, column: str) -> pd.DataFrame:
        """13-quarter ARIMA forecasting"""
        model = ARIMA(self.df[column], order=(1, 1, 1))
        fitted = model.fit()
        forecast = fitted.get_forecast(steps=13)
        return pd.DataFrame({
            "Date": forecast.predicted_mean.index,
            column: forecast.predicted_mean.values,
            "CI_Lower": forecast.conf_int()[:, 0],
            "CI_Upper": forecast.conf_int()[:, 1]
        })

# Example usage
forecaster = BalanceSheetForecaster("jpm_10k_2023.csv")
deposits_forecast = forecaster.forecast_13q("deposits")
deposits_forecast.to_csv("13q_forecast.csv")